# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,-30.96,97,100,3.40,RU,1707683623
1,1,dunedin,-45.8742,170.5036,14.33,69,100,0.45,NZ,1707683651
2,2,braunfels,50.5155,8.3892,9.50,95,98,2.42,DE,1707683766
3,3,port-aux-francais,-49.3500,70.2167,7.73,92,100,11.93,TF,1707683766
4,4,utrik,11.2278,169.8474,26.07,75,100,10.94,MH,1707683767


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_hum = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = 'Humidity' ,
    color = 'City',
    hover_cols = ['City'],
    alpha = 0.5
)

# Display the map
cities_hum


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
criteria_df = city_data_df[(city_data_df['Max Temp'] < 25) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
my_cities = criteria_df.dropna(how='any')

# Display the filtered DataFrame
my_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,rajmahal,25.0500,87.8333,15.46,36,38,1.80,IN,1707683771
11,11,isna,25.2934,32.5540,17.11,43,0,1.67,EG,1707683772
31,31,ma'rib,15.4591,45.3253,23.05,46,23,2.54,YE,1707683789
52,52,fort st. john,56.2499,-120.8529,1.94,48,40,5.14,CA,1707683507
73,73,shajapur,23.4333,76.2667,15.34,39,84,3.29,IN,1707683818


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy() 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] =''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,rajmahal,IN,25.0500,87.8333,36,
11,isna,EG,25.2934,32.5540,43,
31,ma'rib,YE,15.4591,45.3253,46,
52,fort st. john,CA,56.2499,-120.8529,48,
73,shajapur,IN,23.4333,76.2667,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 1



params = {
    'categories':categories,
    'apiKey':geoapify_key,
    limit:limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = my_cities.loc[index, 'Lat']
    longitude = my_cities.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rajmahal - nearest hotel: No hotel found
isna - nearest hotel: No hotel found
ma'rib - nearest hotel: No hotel found
fort st. john - nearest hotel: Fort St. John Motor Inn
shajapur - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
meybod - nearest hotel: هتل رستوران مهربد فیروزه میبد
i-n-salah - nearest hotel: باجودة
laojunmiao - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
serdar - nearest hotel: Chynar Hotel
kidal - nearest hotel: No hotel found
al jumum - nearest hotel: No hotel found
ardakan - nearest hotel: هتل رستوران باغ سرای اردکان
ghat - nearest hotel: فندق تاسيلي
kalaleh - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
ceerigaabo - nearest hotel: Sanaag Hotel
rancho palos verdes - nearest hotel: No hotel found
el rio - nearest hotel: Courtyard by Marriott Oxnard Ventura
rosarito - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
10,rajmahal,IN,25.0500,87.8333,36,No hotel found
11,isna,EG,25.2934,32.5540,43,No hotel found
31,ma'rib,YE,15.4591,45.3253,46,No hotel found
52,fort st. john,CA,56.2499,-120.8529,48,Fort St. John Motor Inn
73,shajapur,IN,23.4333,76.2667,39,No hotel found
80,punta arenas,CL,-53.1500,-70.9167,47,Hotel Lacolet
121,meybod,IR,32.2501,54.0166,23,هتل رستوران مهربد فیروزه میبد
165,i-n-salah,DZ,27.1935,2.4607,31,باجودة
166,laojunmiao,CN,39.8333,97.7333,23,No hotel found
181,khash,IR,28.2211,61.2158,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    scale = 3,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'],
    alpha = 0.5
)

# Display the map
map_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)